In [ ]:
import numpy as np
import boto3
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput

# Initialize
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
processed_bucket = 'cybersec-processed-data-plh92c1q'

print(f"✅ Initialized: {region}")

In [ ]:
# Create bulletproof binary data
np.random.seed(42)
n_samples = 1000
n_features = 10

# Features: normalized floats
X = np.random.uniform(-1, 1, (n_samples, n_features))

# Labels: EXACTLY 0 or 1 (integers)
y = np.random.choice([0, 1], size=n_samples).astype(np.int32)

# Combine: label first, force correct dtypes
data = np.zeros((n_samples, n_features + 1))
data[:, 0] = y.astype(np.int32)  # Force integer labels
data[:, 1:] = X  # Float features

print(f"Data shape: {data.shape}")
print(f"Label column type: {data[:, 0].dtype}")
print(f"Unique labels: {np.unique(data[:, 0])}")
print(f"Label range: [{data[:, 0].min()}, {data[:, 0].max()}]")

# Verify labels are exactly 0 or 1
assert np.all(np.isin(data[:, 0], [0, 1])), "Labels not binary"
print("✅ Data validation passed")

In [ ]:
# CLEAR OLD S3 DATA FIRST
s3 = boto3.client('s3')

# Delete all existing data
for prefix in ['train/', 'validation/', 'test/']:
    objects = s3.list_objects_v2(Bucket=processed_bucket, Prefix=prefix)
    if 'Contents' in objects:
        delete_keys = [{'Key': obj['Key']} for obj in objects['Contents']]
        s3.delete_objects(Bucket=processed_bucket, Delete={'Objects': delete_keys})
        print(f"🗑️ Cleared {prefix}")

# Split and save with integer format
train_data = data[:700]
val_data = data[700:900]
test_data = data[900:]

# Save with integer format for labels
np.savetxt('/tmp/train.csv', train_data, delimiter=',', fmt=['%d'] + ['%.6f']*n_features)
np.savetxt('/tmp/validation.csv', val_data, delimiter=',', fmt=['%d'] + ['%.6f']*n_features)
np.savetxt('/tmp/test.csv', test_data, delimiter=',', fmt=['%d'] + ['%.6f']*n_features)

# Verify saved files
test_load = np.loadtxt('/tmp/train.csv', delimiter=',')
print(f"Loaded labels: {np.unique(test_load[:, 0])}")
print(f"Loaded label type: {test_load[:, 0].dtype}")

# Upload fresh data to S3
s3.upload_file('/tmp/train.csv', processed_bucket, 'train/train.csv')
s3.upload_file('/tmp/validation.csv', processed_bucket, 'validation/validation.csv')
s3.upload_file('/tmp/test.csv', processed_bucket, 'test/test.csv')

print("✅ Fresh data uploaded to S3")

In [ ]:
# Configure XGBoost with minimal settings
container = sagemaker.image_uris.retrieve('xgboost', region, version='1.5-1')

xgb_estimator = Estimator(
    image_uri=container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f's3://{processed_bucket}/model-output/',
    sagemaker_session=sess,
    hyperparameters={
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'num_round': 10,
        'max_depth': 3,
        'eta': 0.3
    }
)

print("✅ XGBoost configured")

In [ ]:
# Train model
train_input = TrainingInput(f's3://{processed_bucket}/train/', content_type='text/csv')
validation_input = TrainingInput(f's3://{processed_bucket}/validation/', content_type='text/csv')

print("🔄 Starting training...")
xgb_estimator.fit({
    'train': train_input,
    'validation': validation_input
})

print("✅ Training completed successfully!")

In [ ]:
# Check training job status before deployment
job_name = xgb_estimator.latest_training_job.name
job_details = sess.sagemaker_client.describe_training_job(TrainingJobName=job_name)
status = job_details['TrainingJobStatus']

print(f"Training job status: {status}")

if status != 'Completed':
    print(f"❌ Training failed: {job_details.get('FailureReason', 'Unknown error')}")
    raise Exception(f"Cannot deploy - training job {status}")

# Deploy model
import time
endpoint_name = f'threat-detection-{int(time.time())}'

predictor = xgb_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    endpoint_name=endpoint_name
)

print(f"✅ Deployed: {predictor.endpoint_name}")

In [ ]:
# Test prediction
test_sample = np.random.uniform(-1, 1, n_features)
csv_input = ','.join(map(str, test_sample))

result = predictor.predict(csv_input, initial_args={'ContentType': 'text/csv'})
prediction = float(result)

print(f"✅ Prediction: {prediction:.4f}")
print(f"✅ Binary: {1 if prediction > 0.5 else 0}")
print(f"✅ Status: {'Attack' if prediction > 0.5 else 'Normal'}")

In [ ]:
print("\n🎉 TRAINING COMPLETED SUCCESSFULLY!")
print(f"📊 Endpoint: {predictor.endpoint_name}")
print(f"🌐 Frontend: http://cybersec-frontend-plh92c1q.s3-website-eu-west-1.amazonaws.com")
print(f"🚀 API: https://ve4dywke7a.execute-api.eu-west-1.amazonaws.com/prod/predict")
print("\nℹ️ Update Lambda manually with: aws lambda update-function-configuration --function-name threat-detection-predict-plh92c1q --environment Variables='{ENDPOINT_NAME=' + predictor.endpoint_name + '}'")